https://wrds-www.wharton.upenn.edu/pages/wrds-research/applications/python-replications/historical-sp-500-index-constituents

https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/stock-security-files/daily-stock-file/

https://wrds-www.wharton.upenn.edu/data-dictionary/contrib_global_factor/global_factor/

In [1]:
import pandas as pd
import wrds

In [2]:
conn = wrds.Connection()

Loading library list...
Done


In [3]:
begdate = '2015-01-01'
enddate = '2022-12-30'

Get S&P 500 Constituent

In [4]:
sp500 = conn.raw_sql(f"""
                        select a.*, b.date
                        from crsp.msp500list as a,
                        crsp.msf as b
                        where a.permno=b.permno
                        and b.date >= a.start and b.date<= a.ending
                        and b.date>='{begdate}'
                        and b.date<='{enddate}'
                        order by date;
                        """, date_cols=['start', 'ending', 'date'])

In [5]:
mse = conn.raw_sql("""
                        select comnam, namedt, nameendt, 
                        permno, ticker
                        from crsp.msenames
                        """, date_cols=['namedt', 'nameendt'])

# if nameendt is missing then set to today date
mse['nameendt']=mse['nameendt'].fillna(pd.to_datetime('today'))

In [6]:
# Merge with SP500 data
sp500_full = pd.merge(sp500, mse, how = 'left', on = 'permno')

# Impose the date range restrictions
sp500_full = sp500_full.loc[(sp500_full.date>=sp500_full.namedt) \
                            & (sp500_full.date<=sp500_full.nameendt)]

In [7]:
sp500 = sp500_full[['permno', 'comnam', 'ticker']].drop_duplicates()
sp500

,permno,comnam,ticker
3,10104.0,ORACLE CORP,ORCL
5,10107.0,MICROSOFT CORP,MSFT
10,10138.0,T ROWE PRICE GROUP INC,TROW
25,10145.0,HONEYWELL INTERNATIONAL INC,HON
35,10147.0,E M C CORP MA,EMC
...,...,...,...
334579,16581.0,INVITATION HOMES INC,INVH
334855,12476.0,TARGA RESOURCES CORP,TRGP
340281,75607.0,GEN DIGITAL INC,GEN
340687,82276.0,ARCH CAPITAL GROUP LTD NEW,ACGL


Read Price Volume data

In [8]:
permno_list = sp500['permno'].astype('int').to_list()
formatted_permnos = ",".join(["'{}'".format(permno) for permno in permno_list])

In [9]:
query = f"""
SELECT 
    permno, date, prc, vol, openprc, askhi, bidlo, ret, shrout
FROM 
    crsp_a_stock.dsf 
WHERE 
    permno IN ({formatted_permnos}) AND date BETWEEN '{begdate}' AND '{enddate}'
"""

price_vol = conn.raw_sql(query)

In [10]:
price_vol['date'] = pd.to_datetime(price_vol['date'])
price_vol['yyyy-mm'] = price_vol['date'].dt.strftime('%Y-%m')
price_vol

,permno,date,prc,vol,openprc,askhi,bidlo,ret,shrout,yyyy-mm
0,10104.0,2015-01-02,44.330002,15070165.0,45.020000,45.189800,43.970001,-0.014232,4391367.0,2015-01
1,10104.0,2015-01-05,43.590000,18375801.0,44.160000,44.250000,43.580002,-0.013986,4391367.0,2015-01
2,10104.0,2015-01-06,43.139999,19229461.0,44.060001,44.180000,42.990002,-0.010323,4391367.0,2015-01
3,10104.0,2015-01-07,43.150002,13502164.0,43.330002,43.520000,43.009998,0.000232,4391367.0,2015-01
4,10104.0,2015-01-08,43.410000,17516877.0,43.630001,43.939999,43.380001,0.006025,4391367.0,2015-01
...,...,...,...,...,...,...,...,...,...,...
221715,93436.0,2022-12-23,123.150002,166917212.0,126.370003,128.617294,121.019997,-0.017551,3157752.0,2022-12
221716,93436.0,2022-12-27,109.099998,208446557.0,117.495003,119.669998,108.760002,-0.114089,3157752.0,2022-12
221717,93436.0,2022-12-28,112.709999,220818648.0,110.349998,116.269997,108.239998,0.033089,3157752.0,2022-12
221718,93436.0,2022-12-29,121.820000,221592714.0,120.385002,123.570000,117.495003,0.080827,3157752.0,2022-12


Read GICS data

In [11]:
query = f"""
SELECT 
    DISTINCT permno, gics, date
FROM 
    contrib_global_factor.global_factor
WHERE
    permno IN ({formatted_permnos}) AND date BETWEEN '{begdate}' AND '{enddate}'
"""

gics = conn.raw_sql(query)

In [12]:
gics['date'] = pd.to_datetime(gics['date'])
gics['yyyy-mm'] = gics['date'].dt.strftime('%Y-%m')
gics.drop(columns='date', inplace=True)
gics

,permno,gics,yyyy-mm
0,87717.0,20301010.0,2019-12
1,19502.0,30101010.0,2020-06
2,44329.0,35101010.0,2021-07
3,60442.0,40101015.0,2018-09
4,24328.0,10102020.0,2015-07
...,...,...,...
58326,92157.0,45203020.0,2018-12
58327,58246.0,40203010.0,2021-07
58328,10516.0,30202010.0,2015-10
58329,89641.0,45202030.0,2015-12


Merge price_vol and GICS information

In [13]:
price_vol_gics = pd.merge(price_vol, gics, how='left', on=['permno', 'yyyy-mm'])
price_vol_gics = pd.merge(price_vol_gics, sp500, how='left', on=['permno'])
price_vol_gics

,permno,date,prc,vol,openprc,askhi,bidlo,ret,shrout,yyyy-mm,gics,comnam,ticker
0,10104.0,2015-01-02,44.330002,15070165.0,45.020000,45.189800,43.970001,-0.014232,4391367.0,2015-01,45103020.0,ORACLE CORP,ORCL
1,10104.0,2015-01-05,43.590000,18375801.0,44.160000,44.250000,43.580002,-0.013986,4391367.0,2015-01,45103020.0,ORACLE CORP,ORCL
2,10104.0,2015-01-06,43.139999,19229461.0,44.060001,44.180000,42.990002,-0.010323,4391367.0,2015-01,45103020.0,ORACLE CORP,ORCL
3,10104.0,2015-01-07,43.150002,13502164.0,43.330002,43.520000,43.009998,0.000232,4391367.0,2015-01,45103020.0,ORACLE CORP,ORCL
4,10104.0,2015-01-08,43.410000,17516877.0,43.630001,43.939999,43.380001,0.006025,4391367.0,2015-01,45103020.0,ORACLE CORP,ORCL
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1397279,93436.0,2022-12-23,123.150002,166917212.0,126.370003,128.617294,121.019997,-0.017551,3157752.0,2022-12,25102010.0,TESLA INC,TSLA
1397280,93436.0,2022-12-27,109.099998,208446557.0,117.495003,119.669998,108.760002,-0.114089,3157752.0,2022-12,25102010.0,TESLA INC,TSLA
1397281,93436.0,2022-12-28,112.709999,220818648.0,110.349998,116.269997,108.239998,0.033089,3157752.0,2022-12,25102010.0,TESLA INC,TSLA
1397282,93436.0,2022-12-29,121.820000,221592714.0,120.385002,123.570000,117.495003,0.080827,3157752.0,2022-12,25102010.0,TESLA INC,TSLA


Save to parquet

In [14]:
price_vol_gics.to_parquet('data/price_vol_gics.parquet.gzip', compression='gzip')

In [ ]:
conn.close()

GICS map

Note that according to GICS 2023 standard, some names have changed.

In [ ]:
sector_map = {
    10: 'Energy',
    15: 'Materials',
    20: 'Industrials',
    25: 'Consumer Discretionary',
    30: 'Consumer Staples',
    35: 'Health Care',
    40: 'Financials',
    45: 'Information Technology',
    50: 'Communication Services',
    55: 'Utilities',
    60: 'Real Estate'
}

In [ ]:
industry_group_map = {
    1010: 'Energy',
    1510: 'Materials',
    2010: 'Capital Goods',
    2020: 'Commercial & Professional Services',
    2030: 'Transportation',
    2510: 'Automobiles & Components',
    2520: 'Consumer Durables & Apparel',
    2530: 'Consumer Services',
    2550: 'Consumer Discretionary Distribution & Retail',
    3010: 'Consumer Staples Distribution & Retail',
    3020: 'Food, Beverage & Tobacco',
    3030: 'Household & Personal Products',
    3510: 'Health Care Equipment & Services',
    3520: 'Pharmaceuticals, Biotechnology & Life Sciences',
    4010: 'Banks',
    4020: 'Financial Services',
    4030: 'Insurance',
    4510: 'Software & Services',
    4520: 'Technology Hardware & Equipment',
    4530: 'Semiconductors & Semiconductor Equipment',
    5010: 'Telecommunication Services',
    5020: 'Media & Entertainment',
    5510: 'Utilities',
    6010: 'Equity Real Estate Investment Trusts (REITs)',
    6020: 'Real Estate Management & Development'
}
